# Practical results and complexities

In [1]:
load('utils/magma.sage')
import re
import ast
from os import getcwd, walk, path, makedirs

resultFiles = {}
dirNames = ["magma","magma/GE"]
for dirname in dirNames:
    resultFiles[dirname] = []
    for (dirPath, dirNames, fileNames) in walk(dirname + "/" + "results"):
        resultFiles[dirname] += [fileName[:-4] for fileName in fileNames]
        break
    resultFiles[dirname].sort()

print(resultFiles)

{'magma': ['two_stage_ca_mono31_t24r16c8d8', 'two_stage_ica_mono31_t24r16c8d8', 'two_stage_nca_tip4_d2_s1_1_t1_4_u_1_m16r12c4d2_alpha7_pffffffff00000001', 'two_stage_nca_tip4_d3_s1_1_t1_4_u_1_m16r12c4d3_alpha7_pffffffff00000001', 'two_stage_sfs_ca_tip4_s1_0_t1_5_u_1_m16r12c4d4_alpha7_pffffffff00000001', 'two_stage_sfs_ca_tip4_s1_1_t1_4_u_1_m16r12c4d4_alpha7_pffffffff00000001', 'two_stage_sfs_ca_tip4p_s1_0_t1_5_u_1_m12r8c4d4_alpha7_pffffffff00000001', 'two_stage_sfs_ca_tip4p_s1_1_t1_4_u_1_m12r8c4d4_alpha7_pffffffff00000001', 'two_stage_sfs_ca_tip5_s1_0_t1_7_u_1_m16r10c6d5_alpha7_pffffffff00000001', 'two_stage_sfs_ca_tip5_s1_1_t1_6_u_1_m16r10c6d5_alpha7_pffffffff00000001'], 'magma/GE': ['ica_mono64_t12r8c4d4', 'ica_mono64_t12r8c4d4_1', 'two_stage_ca_mono31_t24r16c8d8', 'two_stage_ica_mono31_t24r16c8d8', 'two_stage_nca_tip4_d2_s1_1_t1_4_u_1_m16r12c4d2_alpha7_pffffffff00000001']}


In [2]:
def attackInfoTable(resultFiles, target='tip', cols=None, showOnlySols=False, useGelim=True):
    
    tableData = [['attack', 'Permutation', 'Parameters', '$|V|$', 'Time (s)', 'Memory (MB)', '$n_v$','$n_e$', 'degs',
                  '$T_{GB}$','$n_e$', '$d_{\\mathrm{MAC}}$', '$\\mathcal{C}_{GB}^{est}$', '$d_{\\mathrm{max}}$', 
                  '$d_{\\mathrm{exp}}$', '$d_{\\mathrm{reg}}$', '$\\mathcal{C}_{GB}^{exp}$', 'Deg.fall',
                  '$T_{FGLM}$','$n_e$', '$\\mathrm{B}$', '$\\mathcal{C}_{FGLM}^{est}$', '$d_{\\mathcal{I}}$','$\\mathcal{C}_{FGLM}^{exp}$',
                  '$T_{ELIM}$','$d$','$|V|$', 'dirname', 'filename']]
    
    for dirname in resultFiles:
        for filename in resultFiles[dirname]:

            results, _, _ = parse_magma_output(filename, dirname)
            if showOnlySols and results['ELIM']['nsols'] == 0:
                continue

            if 'tip' in filename:
                params = (results['params']['s1'],results['params']['t1'],results['params']['d'])
            elif 'mono' in filename:
                params = None
            else:
                return None

            fp = filename.split('_')
            attack = []
            for word in fp:
                if word.startswith('tip') or word.startswith('mono'):
                    break
                else:
                    attack.append(word)
            attack = '_'.join(attack)

            c = {}
            for alg in ['GB','FGLM']:
                c[alg] = {}
                w = 2.37 if alg == 'GB' else 3
                for mode in results[alg]['c']:
                    k,d = results['gelim']['c'][alg][mode] if useGelim else results[alg]['c'][mode]
                    c[alg][mode] = f"${k:.1f} \\omega + {d:.1f} \\;({round(k*w+d,1)})$" if k is not None else '?'

            degreefall = None
            if results['GB']['dregs'][0] != None: # GB was computed
                if sorted(results['GB']['dregs']) == results['GB']['dregs']:
                    degreefall = False
                else:
                    degreefall = True
            gb_dmax = max(results['GB']['degs']) if results['GB']['degs'][0] != None else None

            #print(results['GB']['dregs'])

            tableData.append([attack, word, params, results['ELIM']['nsols'], results['t'], results['mem'],
                              results['nv'], results['ne'], '-'.join([str(x) for x in results['degs']]),
                              round(results['GB']['t'],2), results['GB']['ne'], results['GB']['dmac'], c['GB']['est'], 
                              gb_dmax, results['GB']['dexp'], results['GB']['dreg'], c['GB']['exp'], degreefall,
                              round(results['FGLM']['t'],2), results['FGLM']['ne'], results['FGLM']['bez'], c['FGLM']['est'], results['FGLM']['vsdim'], c['FGLM']['exp'], 
                              round(results['ELIM']['t'],2), results['ELIM']['unideg'], results['ELIM']['nsols'],
                             dirname, filename])

    #print(attackInfo)
    tableData.sort(key=lambda x: x[-1])
    if cols is not None:
        return table([[elem for i, elem in enumerate(row) if i in cols] for row in tableData])
    return table(tableData,header_row=True)

In [3]:
attackInfoTable(resultFiles, useGelim=True, showOnlySols=True) # rerun to display

attack,Permutation,Parameters,,Time (s),Memory (MB),,,degs,,,,,,,,,Deg.fall,,,,,,,,,,dirname,filename
ica,mono64,,,,,,,2-2-2-2-4-4-4-4,,,,,,,,,,,,,,,,,,,magma/GE,ica_mono64_t12r8c4d4
ica,mono64,,,,,,,2-2-2-2-4-4-4-4,,,,,,,,,,,,,,,,,,,magma/GE,ica_mono64_t12r8c4d4_1
two_stage_ca,mono31,,,,,,,1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2,,,,,,,,,,,,,,,,,,,magma,two_stage_ca_mono31_t24r16c8d8
two_stage_ca,mono31,,,,,,,2-2-2-2-2-2-2-2,,,,,,,,,,,,,,,,,,,magma/GE,two_stage_ca_mono31_t24r16c8d8
two_stage_ica,mono31,,,,,,,1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2,,,,,,,,,,,,,,,,,,,magma,two_stage_ica_mono31_t24r16c8d8
two_stage_ica,mono31,,,,,,,2-2-2-2-2-2-2-2,,,,,,,,,,,,,,,,,,,magma/GE,two_stage_ica_mono31_t24r16c8d8
two_stage_nca,tip4,,,,,,,1-1-1-1-42-42,,,,,,,,,,,,,,,,,,,magma,two_stage_nca_tip4_d2_s1_1_t1_4_u_1_m16r12c4d2_alpha7_pffffffff00000001
two_stage_nca,tip4,,,,,,,42-42,,,,,,,,,,,,,,,,,,,magma/GE,two_stage_nca_tip4_d2_s1_1_t1_4_u_1_m16r12c4d2_alpha7_pffffffff00000001
two_stage_nca,tip4,,,,,,,1-1-1-1-42-42-42,,,,,,,,?,,,,,,,,,,,magma,two_stage_nca_tip4_d3_s1_1_t1_4_u_1_m16r12c4d3_alpha7_pffffffff00000001
two_stage_sfs_ca,tip4,,,,,,,6-6-6-6,,,,,,,,,,,,,,,,,,,magma,two_stage_sfs_ca_tip4_s1_0_t1_5_u_1_m16r12c4d4_alpha7_pffffffff00000001


## Correctness of solutions

In [4]:
import re

def parse_dictionary_string(expr_str, P):
    # Remove the curly braces
    expr_str = expr_str.strip("{}")

    # Split into key-value pairs
    pairs = re.split(r',\s*(?=\w+\s*:\s*)', expr_str)

    parsed_dict = {}
    for pair in pairs:
        key, value = pair.split(":", 1)
        key = key.strip()
        value = value.strip()
        parsed_dict[key] = P(value)
    
    return parsed_dict

## (1) Tip5-like

In [5]:
load('Tip5.sage')

In [6]:
def tip5_recover_colliding_messages(tip, filename, dirname, printTex=False):
    assert('GE' not in dirname) # this is not implemented
    vec2str = lambda v : " ".join([f"{tip.from_field(x):016x}" for x in v])
    
    res, _, P = parse_magma_output(filename, dirname)

    assert(tip.m == res['params']['n'] and tip.r == res['params']['r'] and tip.c == res['params']['c'] and tip.d == res['params']['d'])

    # Parse fixed elements
    [i2f,Di2,Do2] = ast.literal_eval(f"[{res['fixed']}]")
    print(f"s1 = {sum([Di2[i] != 0 for i in range(tip.s)])}")
    
    if res['ELIM']['nsols'] == 0:
        print("No solution found")
        return

    F = P.base_ring()
    i2f = {k: F(v) for k, v in i2f.items()}
    Di2 = vector(F, Di2)
    Do2 = vector(F, Do2)
    
    if filename.startswith("two_stage_ca") or filename.startswith("two_stage_nca"):
        # Calculated solutions for output of second nonlinear layer
        sol_i2 = {str(k) : res['ELIM']['sols'][0][i] for i, k in enumerate(P.gens())}
        
        # Reconstruct input i2 to second nonlinear layer
        i2 = vector(F, tip.m)
        for i in range(tip.m):
            x = f'x{i}'

            if x in i2f and x not in sol_i2:
                # Enter fixed variable values in vector
                i2[i] = i2f[x]
            elif x not in i2f and x in sol_i2:
                # Enter computed variable values in vector
                i2[i] = sol_i2[x]
            else:
                assert(False)
        
        # Make sure differences work out
        I2 = i2 + Di2
        o2 = tip.nonlinear_layer(i2)
        O2 = tip.nonlinear_layer(I2)
        assert(O2 - o2 == Do2)
        
        # Calculate back to input
        i1 = tip.nonlinear_layer(tip.linear_layer(i2, r=0, inv=True), inv=True)
        I1 = tip.nonlinear_layer(tip.linear_layer(I2, r=0, inv=True), inv=True)
        
        m, M = i1[:tip.r], I1[:tip.r]  # message part
        c, C = i1[tip.r:], I1[tip.r:]  # capacity part
        assert(m != M) # Make sure messages are different
        assert(c==C and all([ci == tip.c_value for ci in c])) # Make sure capacity collides and equals initial value

        # Calculate 3r output
        o3r = tip.eval_with_intermediate_states(i1, N=3)[-1]
        O3r = tip.eval_with_intermediate_states(I1, N=3)[-1]
        
        d, D = o3r[:tip.d], O3r[:tip.d]
        assert(d==D) # Make sure digest collides
        
        if printTex:
            collision_to_tex(tip, m, M, iv=c, digest=d, innerCollision=False)
        
        # Print colliding messages and digest
        print('-'*100)        
        print(f"Message 1:\n{vec2str(m)}\n\t-> {vec2str(d)}")
        print(f"Message 2:\n{vec2str(M)}\n\t-> {vec2str(D)}")
        print(f"Initial capacity:\n{vec2str(c)}")
                
        pass
    elif filename.startswith("two_stage_sfs_ca"):
        # Calculated solutions for output of second nonlinear layer
        sol_o2 = {str(k) : res['ELIM']['sols'][0][i] for i, k in enumerate(P.gens())}

        # Reconstruct input i2 to second nonlinear layer
        i2 = vector(F, tip.m)
        for i in range(tip.m):
            x = f'x{i}'
            y = f'y{i}'

            if x in i2f and y not in sol_o2:
                # Enter fixed variable values in vector
                i2[i] = i2f[x]
            elif x not in i2f and y in sol_o2:
                # Enter computed variable values in vector
                y_value = sol_o2[y]
                x_value = tip.S(y_value, inv=True) if i < tip.s else tip.T(y_value, inv=True)
                i2[i] = x_value
            else:
                assert(False)

        # Make sure differences work out
        I2 = i2 + Di2
        o2 = tip.nonlinear_layer(i2)
        O2 = tip.nonlinear_layer(I2)
        assert(O2 - o2 == Do2)

        # Calculate back to input
        i1 = tip.nonlinear_layer(tip.linear_layer(i2, r=0, inv=True), inv=True)
        I1 = tip.nonlinear_layer(tip.linear_layer(I2, r=0, inv=True), inv=True)
        
        m, M = i1[:tip.r], I1[:tip.r]  # message part
        c, C = i1[tip.r:], I1[tip.r:]  # capacity part
        assert(m != M) # Make sure messages are different
        assert(c==C) # Make sure capacity collides

        # Calculate 3r output
        o3r = tip.eval_with_intermediate_states(i1, N=3)[-1]
        O3r = tip.eval_with_intermediate_states(I1, N=3)[-1]
        
        d, D = o3r[:tip.d], O3r[:tip.d]
        assert(d==D) # Make sure digest collides
        
        if printTex:
            collision_to_tex(tip, m, M, iv=c, digest=d, innerCollision=False)
        
        # Print colliding messages and digest
        print('-'*100)
        print(f"Message 1:\n{vec2str(m)}\n\t-> {vec2str(d)}")
        print(f"Message 2:\n{vec2str(M)}\n\t-> {vec2str(D)}")
        print(f"Initial capacity:\n{vec2str(c)}")
        
    else:
        assert(False)

### Two-stage near collision for 3-round Tip4 (Section 5.3)

In [7]:
tip4_d2 = Tip5(name="TIP4", rate=12, capacity=4, digest_length=2)

In [8]:
tip5_recover_colliding_messages(tip4_d2, "two_stage_nca_tip4_d2_s1_1_t1_4_u_1_m16r12c4d2_alpha7_pffffffff00000001", dirname="magma", printTex=False)

s1 = 1
----------------------------------------------------------------------------------------------------
Message 1:
c6302a9416e1e7b3 191740c2110b592d 12b397fb06e33789 29f060477c46d55f 16994cb4ce4f0cc8 076bdd4505177614 44ffcea890f7f274 295542230aec227e 6cc6658831bc4f72 2cc445c05ac6250c 4721e7d54ca12911 a487829e83cdc2a9
	-> ea4442b2c6cde046 94587130d50f9581
Message 2:
1c0032b4ce793a22 721d6b79e9cfb508 c8b06bb2c12f1015 a3458bf3e71280c6 ab4623ee67400e0b 4d3744b090c37448 2425f22fe08cb9d4 830f280903acb5ef 9b62df4108d31f9c b63b522d993f6526 3ed2820678c991ed 06d101604089fa5c
	-> ea4442b2c6cde046 94587130d50f9581
Initial capacity:
0000000000000001 0000000000000001 0000000000000001 0000000000000001


### Two-stage SFS collision for 3-round Tip5 (Section 5.4)

In [9]:
tip5_recover_colliding_messages(tip5, "two_stage_sfs_ca_tip5_s1_0_t1_7_u_1_m16r10c6d5_alpha7_pffffffff00000001", dirname="magma", printTex=False)

s1 = 0
----------------------------------------------------------------------------------------------------
Message 1:
0510053a63393b5e 070b433153c77856 a9df7ece7c71c3b6 dea507128a64faa1 a567a4cd0d87dbc1 e4e0df5a55148b11 189548e9e2d5578e f021f746d17c1717 3c5e5bdf6e27f570 f54e8c4543ed13c8
	-> acb46c393decc891 e3abad309c016a4d 2ffb1a0ba6264680 4ba83933b09c17b5 6fabfd5b7b5ac0e3
Message 2:
ac95d195f2bd7273 967bebc4a6922bf0 4447e4bfc03b125f a8c7a57ed55a84ac 76fda4ee213a7dc1 de3f65a907005d3f ee35e7d8c8ae4ed0 910d60b34fad5797 c9821c0110240935 201466ca5a0de071
	-> acb46c393decc891 e3abad309c016a4d 2ffb1a0ba6264680 4ba83933b09c17b5 6fabfd5b7b5ac0e3
Initial capacity:
63b5be0187fa05b0 8f4db2bfcabd2302 0a5c6021ee35d565 11f40c2b7c8d452b bce318baebb56f57 e98b25e991ed9829


In [10]:
tip5_recover_colliding_messages(tip5, "two_stage_sfs_ca_tip5_s1_1_t1_6_u_1_m16r10c6d5_alpha7_pffffffff00000001", dirname="magma", printTex=False)

s1 = 1
----------------------------------------------------------------------------------------------------
Message 1:
26eb5964959d4ab0 cad9c975306f24e0 98177f6842f00c38 ddd74b036a970659 9ad411438f91715d 046faeda6b29e735 b8ac9c2b222ae282 545b82a9fc2476cb 0f018b2f945b2929 1a5cb332fdc5b5e9
	-> 6d8a1c7fe40d1529 3f22524323d81e68 47f10ca75b5c06b1 063420beb3a362c2 de553aff4fc3ee3a
Message 2:
18c5e6e4c2972a3e 1474c542455b6bcc 094acc7843a932f7 99cfd196e2bea6b8 4d93bca646cd1a3f 3efed9a040c369fa 461170bb87a0fb43 809cdee02fe46526 528e3392abd318fe aa00622dd562d3bb
	-> 6d8a1c7fe40d1529 3f22524323d81e68 47f10ca75b5c06b1 063420beb3a362c2 de553aff4fc3ee3a
Initial capacity:
68368846a6070f14 9dff347850fe68c4 83a3f67d32465999 aef79fa0bf8afd3c 7c628dbec44a9a6c 75672e390c1a1911


### Two-stage SFS collision for 3-round Tip4 (Section 5.4)

In [11]:
tip5_recover_colliding_messages(tip4, "two_stage_sfs_ca_tip4_s1_0_t1_5_u_1_m16r12c4d4_alpha7_pffffffff00000001", dirname="magma", printTex=False)

s1 = 0
----------------------------------------------------------------------------------------------------
Message 1:
f065c68799b9224f 5f6be20c98289f24 2a35d4c999731105 2f336da9b12d8e3b d5bf22405d1d1c6e 62ea5eb72ef241af 71dbff1985f169c4 398e9ff03a7e62a8 6ab41b1ef7261fee ae33d6bed1d6af17 49f320563c9ad5fa 91f85d88dd643c90
	-> 199feda6f6577448 73977eac80d5f0db 7864fb871040f78a 08ac2e5ec2d03347
Message 2:
f99251c7d59303fc 0244d3ac998244b2 5071809bfcedd711 d2587fef54685ba9 bf83eeefc8aafcd7 b2a6864b9ead20c0 e6299df48ae58aba 1fb4938ab0faf420 53d74eab02fe04a9 f690f3967d1c28c8 e0e56a7b656bda5d 8a57287029e36c73
	-> 199feda6f6577448 73977eac80d5f0db 7864fb871040f78a 08ac2e5ec2d03347
Initial capacity:
05455d646a9fb498 5b98a5cdcbf75625 36611580308009e5 f13961bc0bf79a35


In [12]:
tip5_recover_colliding_messages(tip4, "two_stage_sfs_ca_tip4_s1_1_t1_4_u_1_m16r12c4d4_alpha7_pffffffff00000001", dirname="magma", printTex=False)

s1 = 1
----------------------------------------------------------------------------------------------------
Message 1:
35d036ba7ba809c1 7103bd18738f068a 11b00bdb5b308cf3 5b2f0e02fcc024e6 b63ad3b3eab3cf13 1b4c35ebfaaeba79 7328a11ac38731cb a45103afe4d9ef8b 7ae0723ba2a8c3fb b3c2c13dd6138cbd f942a22e32422de8 10599456ce34e359
	-> 49efa9e157d05057 b4b9d58e2d8fbfb0 44d665fa91ebbf5b 61e52e2c50b84e05
Message 2:
ce209ec9f03d93b7 f7277018bcda8ae0 4101626c3d9d6703 40be3cadb429a14e f52eb5dec9580ffd 00b2fa9e0b848135 83becc5184fe63ac ed2bae0e60764667 a2a63ee7beef7a95 22a85331300de161 6f9433cf8d87da25 c17706e35b9dd7a6
	-> 49efa9e157d05057 b4b9d58e2d8fbfb0 44d665fa91ebbf5b 61e52e2c50b84e05
Initial capacity:
c99c2cd661e08182 b9b4a07d23d56a6d fd3246bf7fa086da 14850b035588cd5c


### Two-stage SFS collision for 3-round Tip4' (Section 5.4)

In [13]:
tip5_recover_colliding_messages(tip4p, "two_stage_sfs_ca_tip4p_s1_0_t1_5_u_1_m12r8c4d4_alpha7_pffffffff00000001", dirname="magma", printTex=False)

s1 = 0
----------------------------------------------------------------------------------------------------
Message 1:
f9e8311f95cb2b77 4835b953ea03ed41 0b47521c88b06001 ac8436d8c449aa10 4ae194055c249e8e 41ffa5ff7fc583ca dd37c9a863fe195b ad014cca15abdc41
	-> 4f885efd35b10b46 3ce80cda16752fc8 93a563d23acb5028 a2cc2fdf2b4b0d6f
Message 2:
abf3f56fbab88503 2f73090e75f87967 1144a30cef40aee0 c975f46f94e8a65e 22b84ee8dd304de5 b37f710c695a5ea3 bc57a1d2c071fd0a 16604e2c176495ae
	-> 4f885efd35b10b46 3ce80cda16752fc8 93a563d23acb5028 a2cc2fdf2b4b0d6f
Initial capacity:
84aefeb85e1c736a 99852f9ac85c8be0 aa2ccabaf5e705e1 6206827be78f9cd2


In [14]:
tip5_recover_colliding_messages(tip4p, "two_stage_sfs_ca_tip4p_s1_1_t1_4_u_1_m12r8c4d4_alpha7_pffffffff00000001", dirname="magma", printTex=False)

s1 = 1
----------------------------------------------------------------------------------------------------
Message 1:
7779e34c4e1f6342 f6c132b6be3b5d62 62965410c399e72d af74ab1c8f4a042d 6b0defee1200f91d 4a80800cb567a49f ea2fdf1294d6bce7 cef143212bb36786
	-> 0da7642edabda721 5d954259c8ac9d05 aa9a69fe91c52067 0cb24e34b2db2420
Message 2:
f9f0df61d3997217 edf73600e0efa146 cae46f59fbd927a1 a1bfcf5a1fc436ca e86e8eaa82729ae5 a3cd6c16b151ad0d 28179915d9cfb57f d77ab0f3e0db1024
	-> 0da7642edabda721 5d954259c8ac9d05 aa9a69fe91c52067 0cb24e34b2db2420
Initial capacity:
b856ef40f616f839 f26ea2c8ca1a661f b02893b153c2737b 810256b636dda084


## (2) Monolith

In [15]:
load('Monolith.sage')
Mono31 = Monolith(n=31, eta=None, t=24, u=8, sis=[7, 8, 8, 8], r=16, c=8, d=8, N=2); 
Mono64 = Monolith(n=64, eta=32, t=12, u=4, sis=[8, 8, 8, 8, 8, 8, 8, 8], r=8, c=4, d=4, N=2);

In [16]:
def mono_recover_colliding_messages(mono, filename, dirname, innerCollision=False, printTex=False):
    
    assert('GE' not in dirname or filename.startswith("ica")) # this is not implemented
    
    res, _, P = parse_magma_output(filename,dirname)
    #print(res)

    assert(mono.t == res['params']['n'] and mono.r == res['params']['r'] and mono.c == res['params']['c'] and mono.d == res['params']['d'])
    
    if res['ELIM']['nsols'] == 0:
        print("No solution found")
        return
    
    if filename.startswith("two_stage_ca") or filename.startswith("two_stage_ica"):
        # Parse fixed elements: [i1f,Di1,Do1]
        [i1f,Di1,Do1] = ast.literal_eval(f"[{res['fixed']}]")

        F = P.base_ring()
        i1f = {k: F(v) for k, v in i1f.items()}
        Di1 = vector(F, Di1)
        Do1 = vector(F, Do1)
    
        # Calculated solutions for input of first nonlinear layer
        sol_i1 = {str(k) : res['ELIM']['sols'][0][i] for i, k in enumerate(P.gens())}
        
        # Reconstruct inputs i1, I1 to first nonlinear layer
        i1 = vector(F, mono.t)

        for i in range(mono.t):
            x = f'x{i}'
            if x in i1f and x not in sol_i1:
                # Enter fixed variable values in vector
                i1[i] = i1f[x]
            elif x not in i1f and x in sol_i1:
                # Enter computed variable values in vector
                i1[i] = sol_i1[x]
            else:
                assert(False)
        
        I1 = i1 + Di1
        
        # Make sure differences work out (after nonlinear layer)
        o1 = mono.nonlinear_layer(i1, inv=False)
        O1 = mono.nonlinear_layer(I1, inv=False)
        assert(O1 - o1 == Do1)
        
        # Calculate back to input of hash function
        i0 = mono.concrete(i1, inv=True)
        I0 = mono.concrete(I1, inv=True)
        
        m, M = i0[:mono.r], I0[:mono.r]  # message part
        c, C = i0[mono.r:], I0[mono.r:]  # capacity part
        assert(m != M) # Make sure messages are different
        assert(c==C and all([ci == mono.c_value for ci in c])) # Make sure capacity collides and equals initial value
        
        # Calculate 2r output
        assert(mono.N == 2)
        o2r = mono.eval_with_intermediate_states(i0)[-1]
        O2r = mono.eval_with_intermediate_states(I0)[-1]
        
        if innerCollision:
            d, D = o2r[mono.r:], O2r[mono.r:]
            assert(d==D) # Make sure inner part collides
        else:
            d, D = o2r[:mono.d], O2r[:mono.d]
            assert(d==D) # Make sure digest collides
        
        if printTex:
            collision_to_tex(mono, m, M, iv=c, inner=d if innerCollision else None, digest=d if not innerCollision else None, innerCollision=innerCollision)
        
    elif filename.startswith("two_stage_sfs_ca"):
        print("Not implemented")
    elif filename.startswith("ica") and "GE" in dirname:
        print(filename)
        # Parse fixed elements: [sub_lin]
        #sub_lin = ast.literal_eval(f"{res['fixed']}")
        sub_lin = parse_dictionary_string(res['fixed'], P)
        
        # Calculated solutions for input to permutation
        F = P.base_ring()
        sol = {v : res['ELIM']['sols'][0][i] for i, v in enumerate(P.gens())}
        sol_sub = {v : eq.substitute(sol) for v, eq in sub_lin.items()}
        
        all_sols = {str(key): value for d in (sol, sol_sub) for key, value in d.items()}
        print(all_sols)
        
        i0 = [all_sols[f'x{i}'] for i in range(mono.r)]
        I0 = [all_sols[f'y{i}'] for i in range(mono.r)]
        
        a = mono.eval_with_intermediate_states(i0)
        A = mono.eval_with_intermediate_states(I0)
        
        m, M = a[0][:mono.r], A[0][:mono.r]  # message part
        c, C = a[0][mono.r:], A[0][mono.r:]  # capacity part
        
        if innerCollision:
            d, D = a[-1][mono.r:], A[-1][mono.r:]
            assert(d==D) # Make sure inner part collides
        else:
            d, D = a[-1][:mono.d], A[-1][:mono.d]
            assert(d==D) # Make sure digest collides
        
    else:
        assert(False)
    
    # Print colliding messages and digest/inner part
    print('-'*100)
    if mono.n == 64:
        vec2str = lambda v : " ".join([f"{mono.f2i(x):016x}" for x in v])
    else:
        vec2str = lambda v : " ".join([f"{mono.f2i(x):08x}" for x in v])
    print(f"Message 1:\n{vec2str(m)}\n\t-> {vec2str(d)}")
    print(f"Message 2:\n{vec2str(M)}\n\t-> {vec2str(D)}")
    print(f"Initial capacity:\n{vec2str(c)}")  

### Two-stage (inner) collision for 2-round Monolith-31 (Section 7.2)

In [17]:
mono_recover_colliding_messages(Mono31, "two_stage_ca_mono31_t24r16c8d8", dirname="magma", innerCollision=False, printTex=False)

----------------------------------------------------------------------------------------------------
Message 1:
085d4eca 395d7046 2339d23d 754a3cda 4b15573c 69665539 3d112bb7 7462de81 4a016c90 4ec18a97 091395e3 03f425df 03f06c78 4d314ce8 2cc1337d 6fd6f33a
	-> 27b31297 05d950f6 6d7e3542 4f4d1ea5 3f08d779 320672d8 352502d9 7dccccab
Message 2:
40c29863 353d230b 1ed66c08 4984f9b9 09d339a8 55aec9ea 4d6b18bc 781c5ded 48764be7 68744981 7d6d5301 7d84cc90 0c8909b8 79bd7043 745ec83a 40ce5b67
	-> 27b31297 05d950f6 6d7e3542 4f4d1ea5 3f08d779 320672d8 352502d9 7dccccab
Initial capacity:
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000


In [18]:
mono_recover_colliding_messages(Mono31, "two_stage_ica_mono31_t24r16c8d8", dirname="magma", innerCollision=True, printTex=False)

----------------------------------------------------------------------------------------------------
Message 1:
29c00f10 3b7173a8 79ea0cd6 0d55fe73 04b4aaef 578ce0fd 1945b9f5 5178fbea 6d2b218c 7942abf3 583952bf 6185fe9f 5339a47f 233560f3 0010614c 5b831c47
	-> 7c957753 09b10941 0bdfbfee 51f1a150 56e2ccfb 31650e10 5855422f 67e6d6af
Message 2:
622558a9 3751266d 7586a6a1 6190bb51 43728d5a 43d555ae 299fa6fa 55327b56 6ba000e3 12f56ade 4c930fde 5b16a551 5bd241bf 4fc1844e 47adf609 2c7a8474
	-> 7c957753 09b10941 0bdfbfee 51f1a150 56e2ccfb 31650e10 5855422f 67e6d6af
Initial capacity:
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000


### Simple collision for 2-round Monolith-64 (Section 3.2)

In [19]:
mono_recover_colliding_messages(Mono64, "ica_mono64_t12r8c4d4", dirname="magma/GE", innerCollision=True, printTex=False)

ica_mono64_t12r8c4d4
{'x4': 4970011319244055193, 'y4': 11880298368557803304, 'x5': 15818285778906477951, 'y5': 768616885678760778, 'x6': 13455226260728080241, 'y6': 10386222975197920302, 'x7': 5800889350106245453, 'y7': 6595061277179734452, 'x0': 986202503252341030, 'y0': 2906412479606117335, 'x1': 1314034506681157083, 'y1': 7800723607575273529, 'x2': 2367156207234380924, 'y2': 6092150873207983409, 'x3': 11104298429394550906, 'y3': 10296104728832965207}
----------------------------------------------------------------------------------------------------
Message 1:
0dafb1f39af4f126 123c636bf95605db 20d9d4075c10f47c 9a1a6496a3fd807a 44f906f66d366299 db85d71b6755757f baba90fecb346771 5080e621266de94d
	-> 000cda713a1183c5 881fafb16f7a36a5 7f3ce13953660b23 d018d5dde86dcace
Message 2:
2855a6c059af53d7 6c41bcca84872039 548baaf10ba5cd31 8ee31cb3244efa57 a4df4c6bf38a6728 0aaaacfc373fe34a 902346c472ac3c2e 5b865d4a1a3b69b4
	-> 000cda713a1183c5 881fafb16f7a36a5 7f3ce13953660b23 d018d5dde86dcace
Ini